In [1]:
import torch
import numpy as np

In [2]:
def expand(x):
    output_tensor = torch.zeros((5, 44100))
    for i in range(x.shape[1]):  # Iterate over the second dimension
        start_idx = i * 4410
        end_idx = start_idx + 4410
        output_tensor[:, start_idx:end_idx] = x[:, i].unsqueeze(1)
    return output_tensor

In [3]:
def tournament_selection(population, losses, tournament_size=3):
    """
    Selects two parents using tournament selection.

    Args:
    - population (list of Tensors): The population from which to select parents.
    - losses (list of floats): The loss associated with each individual in the population, serving as a measure of fitness.
    - tournament_size (int): The number of individuals to sample for each tournament.

    Returns:
    - parent1, parent2 (tuple of Tensors): Two selected parents from the population.
    """
    population_size = len(population)

    # Tournament 1
    indices = np.random.choice(range(population_size), size=tournament_size, replace=False)
    tournament_losses = [losses[i] for i in indices]
    winner_index = indices[np.argmin(tournament_losses)]
    parent1 = population[winner_index]

    # Tournament 2
    indices = np.random.choice(range(population_size), size=tournament_size, replace=False)
    tournament_losses = [losses[i] for i in indices]
    winner_index = indices[np.argmin(tournament_losses)]
    parent2 = population[winner_index]

    return parent1, parent2

In [4]:
def crossover(parent1, parent2):
    mask = torch.randint(0, 2, size=parent1.shape, dtype=torch.bool)
    offspring = torch.where(mask, parent1, parent2)
    return offspring

In [5]:
def adaptive_mutation_rate(current_iteration, max_iterations, start_rate=0.1, end_rate=0.01):
    """
    Calculates an adaptive mutation rate that decreases from start_rate to end_rate over time.

    Args:
    - current_iteration (int): The current iteration number (should start from 0).
    - max_iterations (int): The total number of iterations the algorithm will run.
    - start_rate (float): The initial mutation rate at the start of the algorithm.
    - end_rate (float): The final mutation rate at the end of the algorithm.

    Returns:
    - float: The calculated mutation rate for the current iteration.
    """
    # Linear decay
    rate = start_rate - ((start_rate - end_rate) * (current_iteration / max_iterations))
    
    # Ensure the rate never falls below the end_rate
    return max(rate, end_rate)


In [6]:
n_iters = 100
population_size = 16
elite_size = 2  # Number of elites to carry over to the next generation
shape = (5,40)


In [7]:
solution = torch.randn(shape).clamp(0, 1)
solution = torch.where(solution < 0.5, torch.tensor(1), torch.tensor(0))

population = []
for i in range(16):
    candidates = torch.randint_like(solution, low=0, high=1)
    population.append(candidates)
    
best_loss = 10000000000
best_solution = []

for iteration in range(n_iters):
    
    losses = []
    for i in range(16):
        proposed = torch.where(population[i] < 0.5, torch.tensor(1), torch.tensor(0))
        proposed = expand(proposed).unsqueeze(0)
        y = expand(solution).unsqueeze(0)
    
        loss = torch.nn.functional.l1_loss(y, proposed).item()
        losses.append(loss)

    sorted_indices = np.argsort(losses)
    sorted_population = [population[i] for i in sorted_indices]
    sorted_losses = [losses[i] for i in sorted_indices]

    # Update best solution if found
    if sorted_losses[0] < best_loss:
        best_loss = sorted_losses[0]
        best_solution = sorted_population[0]
        print(f"Iteration {iteration}, Loss: {best_loss}")


    # Elitism: Carry over the best solutions unchanged
    new_population = sorted_population[:elite_size]
    
     # Fill the rest of the new population
    while len(new_population) < population_size:
        # Tournament selection for parent selection
        parent1, parent2 = tournament_selection(sorted_population, sorted_losses)

        # Crossover to produce offspring
        offspring1 = crossover(parent1, parent2)
        offspring2 = crossover(parent2, parent1)

        # Adaptive mutation rate
        mutation_rate = adaptive_mutation_rate(iteration, n_iters)

        # Mutation for offspring
        for offspring in [offspring1, offspring2]:
            if len(new_population) < population_size:  # Check if there's still space in the new population
                if torch.rand(1) < mutation_rate:
                    mutation = torch.randint(-1, 2, size=offspring.shape)
                    mutated_offspring = offspring + mutation
                    mutated_offspring = mutated_offspring.clamp(0, 1)
                    new_population.append(mutated_offspring)

    # Update population for the next iteration
    population = new_population


Iteration 0, Loss: 0.3199999928474426
Iteration 6, Loss: 0.30000001192092896
Iteration 8, Loss: 0.2800000011920929
Iteration 18, Loss: 0.25999999046325684
Iteration 21, Loss: 0.23999999463558197
